In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Exercise: Improve convolutional network

Same as the last time: Try to make the network converge faster and reach a higher accuracy. You can vary e.g. these things:

- number of each type of layers
- size of layers
- convolution size
- learning rate
- [optimizer](https://www.tensorflow.org/versions/r0.7/api_docs/python/train.html#optimizers)
- [activation function](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#activation-functions)
- add [max or average pooling](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#pooling)

You can also try regularization by e.g. adding a cost for large weights or applying [dropout](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#dropout). 

In [ ]:
import tensorflow as tf
from time import strftime

# Clear the graph because we might run this
# cell in Jupyter many times, and we don't
# want multiple copies of the graph.
tf.python.framework.ops.reset_default_graph()

inputs = tf.placeholder(tf.float32, [None, 784], name="inputs")
expected_outputs = tf.placeholder(tf.float32, [None, 10], name="expected_outputs")

inputs_4d = tf.reshape(inputs, [-1, 28, 28, 1])

conv1_kernel = tf.Variable(
   tf.random_normal([3, 3, 1, 8], stddev=0.05), tf.float32, name='conv1_kernel')
conv1_biases = tf.Variable(
   tf.random_normal([8], stddev=0.05), tf.float32, name="conv1_biases")

conv2_kernel = tf.Variable(
   tf.random_normal([3, 3, 8, 16], stddev=0.05), tf.float32, name='conv2_kernel')
conv2_biases = tf.Variable(
   tf.random_normal([16], stddev=0.05), tf.float32, name="conv2_biases")

weights1 = tf.Variable(
    tf.random_normal([28/4*28/4*16, 100], stddev=0.05), tf.float32, name="weights1")
biases1 = tf.Variable(
    tf.random_normal([100], stddev=0.05), tf.float32, name="biases1")

weights2 = tf.Variable(
    tf.random_normal([100, 50], stddev=0.05), tf.float32, name="weights2")
biases2 = tf.Variable(
    tf.random_normal([50], stddev=0.05), tf.float32, name="biases2")

weights3 = tf.Variable(tf.zeros([50, 10], tf.float32), name="weights3")
biases3 = tf.Variable(tf.zeros([10]), tf.float32, name="biases3")


with tf.name_scope("convoluted1"):
   convoluted1 = tf.nn.relu(tf.nn.conv2d(inputs_4d, conv1_kernel, strides=[1, 2, 2, 1], padding='SAME') + conv1_biases)
    
with tf.name_scope("convoluted2"):
   convoluted2 = tf.nn.relu(tf.nn.conv2d(convoluted1, conv2_kernel, strides=[1, 2, 2, 1], padding='SAME') + conv2_biases)
   convoluted2_2d = tf.reshape(convoluted2, [-1, 28/4*28/4*16])
    
with tf.name_scope("layer1"):
    layer1_outputs = tf.nn.relu(tf.matmul(convoluted2_2d, weights1) + biases1)

with tf.name_scope("layer2"):
    layer2_outputs = tf.nn.relu(tf.matmul(layer1_outputs, weights2) + biases2)

with tf.name_scope("softmax_layer"):
    outputs = tf.nn.softmax(tf.matmul(layer2_outputs, weights3) + biases3)
    
with tf.name_scope("cross_entropy"):
    cross_entropies = -tf.reduce_sum(expected_outputs*tf.log(outputs), 1)
    mean_cross_entropy = tf.reduce_mean(cross_entropies, 0)

with tf.name_scope("train_step"):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
    train_step = optimizer.minimize(mean_cross_entropy)

cross_entropy_summary = tf.scalar_summary("cross_entropy", mean_cross_entropy)
initialize = tf.initialize_all_variables()

with tf.name_scope("accuracy"):
    labels = tf.argmax(outputs, 1)
    expected_labels = tf.argmax(expected_outputs, 1)
    correct_prediction = tf.equal(labels, expected_labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print "Graph ready!"

def log_cross_entropy(writer, dataset, step):
    images, labels = dataset.next_batch(100)
    value = session.run(cross_entropy_summary,
            {inputs: images, expected_outputs: labels})
    writer.add_summary(value, step)
    writer.flush()
    
session = tf.Session()
time = strftime("%Y-%m-%d_%H:%M:%S"),
train_writer = tf.train.SummaryWriter("../logs/convolutional_%s_train" % time, graph_def=session.graph_def)
validation_writer = tf.train.SummaryWriter("../logs/convolutional_%s_validation" % time)

session.run(initialize)
for step in xrange(1000):
    batch_inputs, batch_outputs = mnist.train.next_batch(100)
    session.run(train_step, {
            inputs: batch_inputs,
            expected_outputs: batch_outputs})

    if step % 100 == 0:
        print
        print session.run(accuracy, {
                inputs: mnist.validation.images,
                expected_outputs: mnist.validation.labels}),    
    
    if step % 10 == 0:
        print ".",
        log_cross_entropy(train_writer, mnist.train, step)
        log_cross_entropy(validation_writer, mnist.validation, step)            

print
print "Training accuracy: %f" % session.run(accuracy, {
        inputs: mnist.train.images,
        expected_outputs: mnist.train.labels})
print "Validation accuracy: %f" % session.run(accuracy, {
        inputs: mnist.validation.images,
        expected_outputs: mnist.validation.labels})
# print "Test accuracy: %f" % session.run(accuracy, {
#         inputs: mnist.test.images,
#         expected_outputs: mnist.test.labels})

train_writer.close()
validation_writer.close()